# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2199659411295,
 'week52high': 146.19,
 'week52low': 55.09,
 'week52highSplitAdjustOnly': 147.84,
 'week52lowSplitAdjustOnly': 53.9,
 'week52change': 0.6880505767077517,
 'sharesOutstanding': 16957031885,
 'float': 0,
 'avg10Volume': 88190855,
 'avg30Volume': 99073196,
 'day200MovingAvg': 123.37,
 'day50MovingAvg': 133.53,
 'employees': 153777,
 'ttmEPS': 3.68,
 'ttmDividendRate': 0.8401151257840529,
 'dividendYield': 0.006509660967416005,
 'nextDividendDate': '0',
 'exDividendDate': '2021-02-05',
 'nextEarningsDate': '2021-01-25',
 'peRatio': 34.55838423796477,
 'beta': 1.154111119959828,
 'maxChangePercent': 50.60827128753096,
 'year5ChangePercent': 4.832667344005607,
 'year2ChangePercent': 2.144374856133064,
 'year1ChangePercent': 0.6602015135751753,
 'ytdChangePercent': -0.020173081222173487,
 'month6ChangePercent': 0.10316404753961247,
 'month3ChangePercent': 0.11055818289180806,
 'month1ChangePercent': -0.01507412292111774,
 'day30ChangeP

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.6602015135751753

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_string)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,132.55,0.530132,N/A
1,AAL,19.05,-0.345183,N/A
2,AAP,166.97,0.174816,N/A
3,AAPL,132.04,0.643344,N/A
4,ABBV,109.76,0.177789,N/A
...,...,...,...,...
500,YUM,109.38,0.032942,N/A
501,ZBH,163.48,-0.003112,N/A
502,ZBRA,507.99,1.050554,N/A
503,ZION,53.70,0.161349,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.28,2.17235,N/A
1,179,FCX,38.32,2.103894,N/A
2,387,PYPL,297.61,1.407464,N/A
3,467,VIAC,65.70,1.275213,N/A
4,24,ALGN,604.71,1.227079,N/A
5,275,LB,51.81,1.179554,N/A
6,128,DE,344.00,1.058606,N/A
7,502,ZBRA,507.99,1.050554,N/A
8,266,KLAC,344.95,1.040029,N/A
9,410,SIVB,544.40,0.999846,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe

/home/aky/Code/algorithmic-trading-python/env/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.28,2.17235,522
1,179,FCX,38.32,2.103894,521
2,387,PYPL,297.61,1.407464,67
3,467,VIAC,65.70,1.275213,304
4,24,ALGN,604.71,1.227079,33
5,275,LB,51.81,1.179554,386
6,128,DE,344.00,1.058606,58
7,502,ZBRA,507.99,1.050554,39
8,266,KLAC,344.95,1.040029,57
9,410,SIVB,544.40,0.999846,36


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [22]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,132.71,N/A,0.534145,N/A,0.31603,N/A,0.151118,N/A,-0.009902,N/A,N/A
1,AAL,19.14,N/A,-0.352619,N/A,0.503606,N/A,0.510817,N/A,0.158065,N/A,N/A
2,AAP,169.46,N/A,0.179026,N/A,0.058377,N/A,0.153894,N/A,0.004987,N/A,N/A
3,AAPL,130.28,N/A,0.665431,N/A,0.103102,N/A,0.111777,N/A,-0.015472,N/A,N/A
4,ABBV,109.95,N/A,0.180219,N/A,0.13228,N/A,0.05395,N/A,-0.067758,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.20,N/A,0.032275,N/A,0.122815,N/A,0.010907,N/A,-0.035821,N/A,N/A
501,ZBH,163.27,N/A,-0.003001,N/A,0.151412,N/A,0.075335,N/A,-0.042385,N/A,N/A
502,ZBRA,511.34,N/A,1.068551,N/A,0.804573,N/A,0.398863,N/A,0.211227,N/A,N/A
503,ZION,52.20,N/A,0.163361,N/A,0.638736,N/A,0.361771,N/A,0.080311,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [23]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0
            
for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,132.71,N/A,0.534145,0.893069,0.31603,0.651485,0.151118,0.631683,-0.009902,0.348515,N/A
1,AAL,19.14,N/A,-0.352619,0.023762,0.503606,0.809901,0.510817,0.968317,0.158065,0.944554,N/A
2,AAP,169.46,N/A,0.179026,0.621782,0.058377,0.247525,0.153894,0.637624,0.004987,0.447525,N/A
3,AAPL,130.28,N/A,0.665431,0.938614,0.103102,0.332673,0.111777,0.530693,-0.015472,0.30099,N/A
4,ABBV,109.95,N/A,0.180219,0.623762,0.13228,0.409901,0.05395,0.39604,-0.067758,0.089109,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.20,N/A,0.032275,0.407921,0.122815,0.388119,0.010907,0.281188,-0.035821,0.19802,N/A
501,ZBH,163.27,N/A,-0.003001,0.334653,0.151412,0.439604,0.075335,0.461386,-0.042385,0.176238,N/A
502,ZBRA,511.34,N/A,1.068551,0.988119,0.804573,0.940594,0.398863,0.924752,0.211227,0.978218,N/A
503,ZION,52.20,N/A,0.163361,0.594059,0.638736,0.881188,0.361771,0.893069,0.080311,0.79802,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [24]:
from statistics import mean

for row in hqm_dataframe.index:
    momemtum_percentile = []
    for time_period in time_periods:
        momemtum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momemtum_percentile)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,132.71,N/A,0.534145,0.893069,0.31603,0.651485,0.151118,0.631683,-0.009902,0.348515,0.631188
1,AAL,19.14,N/A,-0.352619,0.023762,0.503606,0.809901,0.510817,0.968317,0.158065,0.944554,0.686634
2,AAP,169.46,N/A,0.179026,0.621782,0.058377,0.247525,0.153894,0.637624,0.004987,0.447525,0.488614
3,AAPL,130.28,N/A,0.665431,0.938614,0.103102,0.332673,0.111777,0.530693,-0.015472,0.30099,0.525743
4,ABBV,109.95,N/A,0.180219,0.623762,0.13228,0.409901,0.05395,0.39604,-0.067758,0.089109,0.379703
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.20,N/A,0.032275,0.407921,0.122815,0.388119,0.010907,0.281188,-0.035821,0.19802,0.318812
501,ZBH,163.27,N/A,-0.003001,0.334653,0.151412,0.439604,0.075335,0.461386,-0.042385,0.176238,0.35297
502,ZBRA,511.34,N/A,1.068551,0.988119,0.804573,0.940594,0.398863,0.924752,0.211227,0.978218,0.957921
503,ZION,52.20,N/A,0.163361,0.594059,0.638736,0.881188,0.361771,0.893069,0.080311,0.79802,0.791584


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [25]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,65.12,N/A,1.229974,0.992079,1.437033,0.994059,0.858346,0.992079,0.449726,0.99802,0.994059
1,FCX,38.50,N/A,2.176655,1.0,1.602459,0.99604,0.782471,0.990099,0.216029,0.984158,0.992574
2,DISCA,52.04,N/A,0.719575,0.956436,1.324929,0.988119,1.00457,1.0,0.397985,0.99604,0.985149
3,TWTR,72.29,N/A,0.884603,0.972277,0.86197,0.954455,0.642641,0.982178,0.522986,1.0,0.977228
4,SIVB,540.50,N/A,1.034242,0.986139,1.254375,0.986139,0.519633,0.970297,0.172816,0.950495,0.973267
5,DISCK,44.30,N/A,0.511403,0.885149,1.133565,0.982178,0.871613,0.994059,0.337234,0.992079,0.963366
6,ZBRA,511.34,N/A,1.068551,0.988119,0.804573,0.940594,0.398863,0.924752,0.211227,0.978218,0.957921
7,AMAT,123.54,N/A,0.852873,0.966337,0.920684,0.966337,0.565387,0.972277,0.110659,0.889109,0.948515
8,PYPL,301.24,N/A,1.372755,0.99604,0.449594,0.780198,0.498159,0.962376,0.18306,0.960396,0.924752
9,TPR,40.73,N/A,0.390627,0.815842,1.781667,0.99802,0.402516,0.930693,0.175148,0.952475,0.924257


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [79]:
portfolio_input()

Enter the value of your portfolio:1000000


In [26]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)

for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe

/home/aky/Code/algorithmic-trading-python/env/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,65.12,307,1.229974,0.992079,1.437033,0.994059,0.858346,0.992079,0.449726,0.99802,0.994059
1,FCX,38.50,519,2.176655,1.0,1.602459,0.99604,0.782471,0.990099,0.216029,0.984158,0.992574
2,DISCA,52.04,384,0.719575,0.956436,1.324929,0.988119,1.00457,1.0,0.397985,0.99604,0.985149
3,TWTR,72.29,276,0.884603,0.972277,0.86197,0.954455,0.642641,0.982178,0.522986,1.0,0.977228
4,SIVB,540.50,37,1.034242,0.986139,1.254375,0.986139,0.519633,0.970297,0.172816,0.950495,0.973267
5,DISCK,44.30,451,0.511403,0.885149,1.133565,0.982178,0.871613,0.994059,0.337234,0.992079,0.963366
6,ZBRA,511.34,39,1.068551,0.988119,0.804573,0.940594,0.398863,0.924752,0.211227,0.978218,0.957921
7,AMAT,123.54,161,0.852873,0.966337,0.920684,0.966337,0.565387,0.972277,0.110659,0.889109,0.948515
8,PYPL,301.24,66,1.372755,0.99604,0.449594,0.780198,0.498159,0.962376,0.18306,0.960396,0.924752
9,TPR,40.73,491,0.390627,0.815842,1.781667,0.99802,0.402516,0.930693,0.175148,0.952475,0.924257


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [27]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name="Momentum Strategy", index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [28]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [29]:
column_formats = { 
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 22 , column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [30]:
writer.save()